In [1]:
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
from collections import Counter
import graphviz

In [66]:
os.chdir("E:/Kaggle_Titanic")
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

In [3]:
train['Age'].fillna(train['Age'].mean(),inplace=True)
test['Age'].fillna(test['Age'].mean(),inplace=True)

In [4]:
train['Embarked'].ffill(inplace=True) 
test['Embarked'].ffill(inplace=True)

In [5]:
train['Embarked']=train['Embarked'].map({'S': 1, 'C': 2, 'Q': 3})
test['Embarked']=test['Embarked'].map({'S': 1, 'C': 2, 'Q': 3})

In [6]:
train['Male']=(train['Sex']=='male')
train['Male']=train['Male'].map({True:1,False:0})
train=train.drop(['PassengerId','Name','Fare','Ticket','Cabin','Sex'],axis=1)
test['Male']=(test['Sex']=='male')
test['Male']=test['Male'].map({True:1,False:0})
test_x=test.drop(['PassengerId','Name','Fare','Ticket','Cabin','Sex'],axis=1)

In [7]:
train=train.values
test_x=test_x.values
train_x=train[:,1:]
train_y=train[:,0]

In [8]:
from sklearn.preprocessing import StandardScaler   
scaler = StandardScaler()   
scaler.fit(train_x) 
train_x = scaler.transform(train_x)   
test_x = scaler.transform(test_x) 

In [14]:
from keras import models
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification

C:\Users\huimi\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\huimi\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\huimi\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\huimi\Anaconda3\lib\site-packages\tensorflow\python\framework\

In [34]:
def create_network():
    network = models.Sequential()
    network.add(layers.Dense(units=50, activation='relu', input_shape=(6,)))
    network.add(layers.Dense(units=50, activation='relu'))
    network.add(layers.Dense(units=1, activation='sigmoid'))
    network.compile(loss='binary_crossentropy', 
                    optimizer='adam', 
                    metrics=['accuracy']) 
    return network

In [39]:
neural_network = KerasClassifier(build_fn=create_network, 
                                 epochs=10, 
                                 #batch_size=100, 
                                 verbose=0)

In [40]:
cross_val_score(neural_network, train_x, train_y, cv=3)

array([0.8013468 , 0.82154882, 0.83501684])

In [33]:
network=create_network()

In [41]:
network.fit(train_x,train_y,epochs=10)

Epoch 1/10
891/891 [==============================] - 1s 1ms/step - loss: 0.6464 - acc: 0.6801
Epoch 2/10
891/891 [==============================] - 0s 76us/step - loss: 0.5289 - acc: 0.8204
Epoch 3/10
891/891 [==============================] - 0s 72us/step - loss: 0.4566 - acc: 0.8204
Epoch 4/10
891/891 [==============================] - 0s 77us/step - loss: 0.4271 - acc: 0.8171
Epoch 5/10
891/891 [==============================] - 0s 85us/step - loss: 0.4173 - acc: 0.8171
Epoch 6/10
891/891 [==============================] - 0s 94us/step - loss: 0.4121 - acc: 0.8193
Epoch 7/10
891/891 [==============================] - 0s 86us/step - loss: 0.4083 - acc: 0.8249
Epoch 8/10
891/891 [==============================] - 0s 77us/step - loss: 0.4058 - acc: 0.8249
Epoch 9/10
891/891 [==============================] - 0s 69us/step - loss: 0.4022 - acc: 0.8316
Epoch 10/10
891/891 [==============================] - 0s 58us/step - loss: 0.3994 - acc: 0.8339


In [42]:
predictions=network.predict([test_x])

In [51]:
predictions=(predictions>=0.5).flatten()

In [68]:
test['Survived']=predictions

In [69]:
test['Survived']=test['Survived'].map({True:1,False:0})

In [70]:
test_submission=test[["PassengerId",'Survived']]

In [71]:
test_submission.to_csv("nnw_submission.csv",index=False)

The score on Kaggle is 0.79425. Although the highest score on Kaggle is 1, I feel that it's impossilbe to get that accuracy since there's no strong pattern of features to determin whether a passenger can survive. 